# Analysis of the Police Figures and Crime in Scotland



## Dataset
![Scottish Statistics Logo](https://images.squarespace-cdn.com/content/v1/600e7b160eec4a7d0511f762/1620835698639-NAWDXF41R9FBJEZ8VZ97/SG+logo.png)

In this example we will be using a crime dataset from the Scottish Statistics Website located here - [Scottish Government Statistics](https://statistics.gov.scot/home) <br>
The Scottish Crime Dataset contains data spanning from 1996 to 2023. <br>
Other datasets from the Scottish Government will be used including Poverty Data, Scottish Police Numbers <br>

Data can be found directly from the following links:<br>
https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Frecorded-crime <br>
https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpoverty-all-people <br>
https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpolice-officer-quarterly-strength <br>

In [1]:
import pandas as pd
import numpy as np
import os

## Importing data
Data will be directly read as csv from the http source as shown above.

In [2]:
df_crime = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Frecorded-crime')
print('df_crime created')
#df_poverty = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpoverty-all-people')
#print('df_poverty created')
#df_police = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpolice-officer-quarterly-strength')
#print('df_police created')

df_crime created


## Data Pre-Processing

### Set Data Types and Rename Columns

In [3]:
df_crime = df_crime.astype({'FeatureCode': 'object', 
                            'FeatureName': 'object', 
                            'FeatureType':'object', 
                            'DateCode':'object',
                            'Measurement':'object',
                            'Units':'object',
                            'Value':'int64',
                            'Crime or Offence':'object'})

df_crime.rename(columns={   'FeatureCode': 'code', 
                            'FeatureName': 'area', 
                            'FeatureType':'area_type', 
                            'DateCode':'date',
                            'Measurement':'measurement',
                            'Units':'units',
                            'Value':'value',
                            'Crime or Offence':'crime_category'},
                             inplace=True)

df_crime.dtypes

code              object
area              object
area_type         object
date              object
measurement       object
units             object
value              int64
crime_category    object
dtype: object

### Pivot Measurement Column

In [4]:
df_crime_pivoted = pd.pivot_table(data = df_crime,
                    index = ['code', 'area', 'area_type', 'date', 'crime_category'],
                    columns = ['measurement'],
                    values = 'value')

df_crime_pivoted.columns.name= None
df_crime_pivoted.reset_index(inplace=True)

df_crime = df_crime_pivoted

df_crime.rename(columns={'Count':'count', 'Ratio':'count_per_10k'}, inplace=True)

In [5]:
df_crime.head()

,code,area,area_type,date,crime_category,count,count_per_10k
0,S12000005,Clackmannanshire,Council Area,1996/1997,All Crimes,2852.0,587.0
1,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 1: Non-sexual crimes of violence,581.0,120.0
2,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 2: Sexual crimes,24.0,5.0
3,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 3: Crimes of dishonesty,1245.0,256.0
4,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 4: Damage and reckless behaviour,588.0,121.0


### Remove Aggregate Crime Category Rows

In [6]:
df_crime = df_crime[~df_crime['crime_category'].str.startswith('All ')]

df_crime = df_crime.sort_values(by=['date','area','count_per_10k', 'count'], 
                                ascending=[True, True, False, False])

### Split Date Column

In [7]:
df_crime[['date_start', 'date_end']] = df_crime['date'].str.split('/', n=1, expand=True)
df_crime.drop(columns=['date'], inplace=True)
df_crime = df_crime.astype({'date_start':'int64', 'date_end':'int64'})

### Remove Scotland Rows

In [8]:
df_crime_scotland = df_crime[df_crime['area'] == 'Scotland']

df_crime.drop(df_crime[df_crime['area'] == 'Scotland'].index, inplace=True)

### Bin Count Categories

In [9]:
count_10k_bins = np.linspace(min(df_crime['count_per_10k']), max(df_crime['count_per_10k']), 8)

count_10k_bins_names = ['Minimal', 'Very Low', 'Low', 'Medium', 'High', 'Very High', 'Extremely High']

df_crime['count_category_historic'] = pd.cut(df_crime['count_per_10k'], 
                                             count_10k_bins, 
                                             labels=count_10k_bins_names, 
                                             include_lowest=True)

### Bin Count by Year

In [10]:
# WIP - Need to bin categories by a subset for each year
# such that each row is categories in the year it falls
# Progress - have created mappings for the binning of each year

mapping_bins_annual = pd.DataFrame(columns=['date_start', 'annual_bins'])
df_crime_concatted = df_crime.copy()

for year in df_crime['date_start'].unique():
    df_temp = df_crime[df_crime['date_start'] == year]
    count_10k_bins = np.linspace(min(df_temp['count_per_10k']), max(df_temp['count_per_10k']), 8)
    data = {'date_start':year, 'annual_bins':[count_10k_bins]}
    df_insert = pd.DataFrame(data)
    mapping_bins_annual = pd.concat([mapping_bins_annual, df_insert])

mapping_bins_annual

,date_start,annual_bins
0,1996,"[0.0, 40.42857142857143, 80.85714285714286, 12..."
0,1997,"[0.0, 40.714285714285715, 81.42857142857143, 1..."
0,1998,"[0.0, 46.142857142857146, 92.28571428571429, 1..."
0,1999,"[0.0, 48.285714285714285, 96.57142857142857, 1..."
0,2000,"[0.0, 44.857142857142854, 89.71428571428571, 1..."
0,2001,"[0.0, 42.714285714285715, 85.42857142857143, 1..."
0,2002,"[0.0, 39.57142857142857, 79.14285714285714, 11..."
0,2003,"[0.0, 42.285714285714285, 84.57142857142857, 1..."
0,2004,"[0.0, 45.857142857142854, 91.71428571428571, 1..."
0,2005,"[0.0, 51.285714285714285, 102.57142857142857, ..."


In [11]:
df_crime = df_crime.reset_index(drop=True)

In [12]:
df_crime.describe(include='all')

,code,area,area_type,crime_category,count,count_per_10k,date_start,date_end,count_category_historic
count,33983,33983,33983,33983,33983.000000,33983.000000,33983.000000,33983.000000,33983
unique,32,32,1,50,NaN,NaN,NaN,NaN,7
top,S12000033,Aberdeen City,Council Area,Crimes: Group 3: Other theft,NaN,NaN,NaN,NaN,Minimal
freq,1062,1062,33983,864,NaN,NaN,NaN,NaN,29595
mean,NaN,NaN,NaN,NaN,459.971191,24.415737,2010.454198,2011.454198,NaN
std,NaN,NaN,NaN,NaN,1178.436115,40.780576,7.714958,7.714958,NaN
min,NaN,NaN,NaN,NaN,0.000000,0.000000,1996.000000,1997.000000,NaN
25%,NaN,NaN,NaN,NaN,21.000000,2.000000,2004.000000,2005.000000,NaN
50%,NaN,NaN,NaN,NaN,91.000000,7.000000,2011.000000,2012.000000,NaN
75%,NaN,NaN,NaN,NaN,378.000000,26.000000,2017.000000,2018.000000,NaN


In [13]:
df_crime['count_category_historic'].value_counts(normalize=True).round(5) * 100

count_category_historic
Minimal           87.088
Very Low           8.775
Low                2.901
Medium             0.859
High               0.282
Very High          0.082
Extremely High     0.012
Name: proportion, dtype: float64

In [14]:
df_crime.sample(20)

,code,area,area_type,crime_category,count,count_per_10k,date_start,date_end,count_category_historic
29254,S12000034,Aberdeenshire,Council Area,Crimes: Group 3: Other dishonesty,85.0,3.0,2020,2021,Minimal
15870,S12000039,West Dunbartonshire,Council Area,Crimes: Group 4: Reckless conduct,133.0,15.0,2010,2011,Minimal
7736,S12000026,Scottish Borders,Council Area,Crimes: Group 1: Serious assault and attempted...,81.0,7.0,2003,2004,Minimal
747,S12000038,Renfrewshire,Council Area,Offences: Group 6: Threatening and abusive beh...,2358.0,134.0,1996,1997,Low
8412,S12000017,Highland,Council Area,Crimes: Group 3: Theft of a motor vehicle,258.0,12.0,2004,2005,Minimal
3265,S12000045,East Dunbartonshire,Council Area,Offences: Group 8: Driving under the influence,136.0,13.0,1999,2000,Minimal
32367,S12000040,West Lothian,Council Area,Offences: Group 8: Mobile phone offences,45.0,2.0,2021,2022,Minimal
7181,S12000042,Dundee City,Council Area,Crimes: Group 4: Reckless conduct,119.0,8.0,2003,2004,Minimal
15999,S12000034,Aberdeenshire,Council Area,Crimes: Group 4: Reckless conduct,115.0,5.0,2011,2012,Minimal
22666,S12000048,Perth and Kinross,Council Area,Crimes: Group 1: Robbery,12.0,1.0,2015,2016,Minimal
